In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

baseurl = 'https://www.noon.com'

# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
# }

product_links = []

for x in range(1,65):
    r = requests.get(f'https://www.noon.com/egypt-en/sports-and-outdoors/exercise-and-fitness/yoga-16328/?limit=50&page={x}')
    soup = BeautifulSoup(r.content)
    spans =  soup.find_all('span', {'class': 'sc-5e739f1b-0 gEERDr wrapper productContainer'})
    for span in spans:
        links = span.find_all('a')
        for link in links:
            productlinks = baseurl + link['href']
            product_links.append(productlinks)

prdctlist = []
for links in product_links:
    r = requests.get(links)
    soup = BeautifulSoup(r.content)
    name = soup.find('h1' , class_ = 'sc-c44e3e2d-12 kODUYj').text.strip()
    brand = soup.find('div' , class_ = 'sc-c44e3e2d-11 jbKJDi').text.strip()
    salesprice = soup.find('div' , class_ = 'priceNow').text.strip()
    price = salesprice.split(u'\xa0')
    storename = soup.find('span' , class_ = 'allOffers').text.strip()
    try:
        prtnrrating = soup.find('div' , class_ = 'sc-d711b2ac-5 hKZAQq').text.strip()
    except:
        prtnrrating = "none"
    date_object = datetime.date.today()
    try:
        sellerrating = soup.find('div' , class_ ='sc-d711b2ac-4 bQMNCl').text.strip()
    except:
        sellerrating = "none"
    try:
        ogprice = soup.find('div' , class_ = 'priceWas').text.strip()
    except:
        ogprice = 'none'
    try:
        sellercount = soup.find('div' , class_ = 'sc-d711b2ac-5 hKZAQq').text.strip()
    except:
        sellercount = 'none'
    link5 = links
    linkprdct = links.split("/")
    ean = linkprdct[5]
    position1 = soup.find('div' , class_ = 'sc-e4a456cc-5 hsjerl').text.strip()

    try:
        averagerating = soup.find('div' , class_ = 'sc-c44e3e2d-0 kLXCWc').text.strip()
    except:
        averagerating = "none"
    try:
        prdctrating = soup.find('div', class_ = 'sc-31ebbe94-0 cqbyTR').text.strip()
    except:
        prdctrating = 'none'

    noondata = {
        'Date':date_object,
        'Product Name': name,
        'Brand': brand,
        'Currency': price[0],
        'Store Name': storename,
        'Partner Rating' : sellerrating,
        'Partner Rating Count': prtnrrating.replace('% Positive Ratings' , ''),
        'Sales Price' : price[1],
        'Price' : ogprice.replace("EGP " , ''),
        'Link' : link5,
        'EAN': linkprdct[5],  
        'Position1':position1.replace(' & ','/'),
        'ratings':prdctrating
        }
    
    prdctlist.append(noondata)
    print('Saving: ', noondata['Product Name'])
    
df = pd.DataFrame(prdctlist)
df.to_excel('A2C.xlsx')
print(df.head(15))

In [66]:
data = pd.read_excel('A2C.xlsx')
data.shape

(3200, 14)

In [60]:
df = pd.read_excel('A2C.xlsx' , na_values = 'none')

In [62]:
dt = pd.DataFrame(df)

In [63]:
#Most expensive product
dt.max()

Unnamed: 0                                                           3199
Date                                                  2022-10-16 00:00:00
Product Name                                             Yoga Roller 30cm
Brand                                                               kings
Currency                                                              EGP
Store Name                                                          vogue
Partner Rating                                                        3.8
Partner Rating Count                                                 89.0
Sales Price                                                         799.0
Price                                                              1049.0
Link                    https://www.noon.com/egypt-en/yoga-roller-30cm...
EAN                                                ZFEE22B4CAE429FFBB22DZ
Position1               Sports, Fitness/OutdoorsExercise/FitnessYogaYo...
ratings                               

In [64]:
#Cheapest Product 
dt.min()

Unnamed: 0                                                              0
Date                                                  2022-10-16 00:00:00
Product Name            11-Piece Yoga Resistance Belt With Bag 22x6x19...
Brand                                                        Athlete Home
Currency                                                              EGP
Store Name                                                 Alfayez sports
Partner Rating                                                        2.9
Partner Rating Count                                                 29.0
Sales Price                                                          59.9
Price                                                               250.0
Link                    https://www.noon.com/egypt-en/11-piece-yoga-re...
EAN                                                            N16529626A
Position1               Sports, Fitness/OutdoorsExercise/FitnessYogaBl...
ratings                               

In [55]:
#Number of Products From Each Brand
dt['Brand'].value_counts()

Athlete Home      1664
BODY SCULPTURE     256
UFIT               256
Generic            192
Spurt              128
Gymbit             128
Prickly Pear        64
LIXADA              64
body heart          64
Mesuca              64
TOEON               64
Liveup              64
Cactus Fitness      64
SportQ              64
kings               64
Name: Brand, dtype: int64

In [56]:
#Number of Products by each seller
dt['Store Name'].value_counts()

Athlete Home         1728
noon                  384
Alfayez sports        320
Kanteen.store         128
Brenda Box            128
Bekstore              128
vogue                  64
Vision Technology      64
Souqegypt              64
Happy shopping         64
Cactus Fitness         64
TBE                    64
Name: Store Name, dtype: int64